In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
st = pd.read_csv("calidad_del_aire_cyl_1997_2013.csv", delimiter=";", index_col = "DIA", parse_dates = True)
st.head()

In [ ]:
#Obtenemos la lista de gases
st = st.sort_values(by='DIA')
gas_headers = list(st)
gas_headers.remove("PROVINCIA")
gas_headers.remove("ESTACIÓN")
#gas_headers.remove("DIA")
gas_headers

In [ ]:
#Aplicamos un describe a cada gas para ver nulos o informacion no numerica
for i in gas_headers:
    gas = pd.Categorical(st[i])
    print(gas.describe())

In [ ]:
#Preparamos los gases para ver su comportamiento a lo largo de tiempo, se puede observar que 
#estos gases tienen un comportamiento no estacional y han descendido a lo largo de tiempo
for i in gas_headers:
    st[i] = st[i].replace("No cumple el anexo IV de la Decisión del Consejo 97/101/CE",0)
    st[i] = st[i].fillna(0)
    st[i] = st[i].astype(float)

st.plot(figsize=(20,10), linewidth=5, fontsize=20)

In [ ]:
for i in gas_headers:
    gas = pd.Categorical(st[i])
    print(gas.describe())

In [ ]:
#Aplicamos un smooth a los datos por año
tsd = st.groupby(pd.Grouper(freq='M')).agg('mean')

In [ ]:
tsd.plot()

In [ ]:
#Usaremos ARIMA porque el valor esta mas cercano al 0 y confirmar la correlacion
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
sm.stats.durbin_watson(tsd)

In [ ]:
#Obtenemos las provincias y estaciones para plotear cada estacion, existen estaciones con el nombre incorrecto, asi que primero corregimos
zonas = pd.Categorical(st['PROVINCIA']+';'+st['ESTACIÓN'])
zonas_unicos = zonas.categories.unique()
print(zonas_unicos)

In [ ]:
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Burgos1','Burgos 1')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Burgos4','Burgos 4')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Burgos5','Burgos 5')

st['ESTACIÓN'] = st['ESTACIÓN'].replace('Miranda de Ebro1','Miranda de Ebro 1')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Miranda de Ebro2','Miranda de Ebro 2')

st['ESTACIÓN'] = st['ESTACIÓN'].replace('Le�n1','Le�n 1')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Leon3','Le�n 3')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Leon 4','Le�n 4')

st['ESTACIÓN'] = st['ESTACIÓN'].replace('Ponferrada4','Ponferrada 4')

st['ESTACIÓN'] = st['ESTACIÓN'].replace('Salamanca4','Salamanca 4')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Salamanca5','Salamanca 5')
st['ESTACIÓN'] = st['ESTACIÓN'].replace('Salamanca6','Salamanca 6')

In [ ]:
#Obtenemos las provincias y estaciones
zonas = pd.Categorical(st['PROVINCIA']+';'+st['ESTACIÓN'])
zonas_unicos = zonas.categories.unique()
print(zonas_unicos)

In [ ]:
#Para usar todos los gases, contabilizamos y sumamos en una columna total
st['TOTAL'] = st.sum(axis=1)

In [ ]:
st

In [ ]:
tsdT = st["TOTAL"].groupby(pd.Grouper(freq='M')).agg('mean')

In [ ]:
#Podemos ver que la serie de tiempo con el nuevo valor de total permanece constante
tsdT.plot()

In [ ]:
#Usaremos ARIMA porque el valor esta mas cercano al 0 y confirmar la correlacion
sm.stats.durbin_watson(tsdT)

In [ ]:
for i in zonas_unicos:
    x = i.split(';')
    print(x[0])
    dtt = st[(st['PROVINCIA'] == x[0]) & (st['ESTACIÓN'] == x[1])]
    tsdTP = dtt['TOTAL'].groupby(pd.Grouper(freq='3M')).agg('mean')
    tsdTP.interpolate(method='linear', inplace=True)
    model = ARIMA(tsdTP , order=(1,0,0))
    model_fit = model.fit()
    pred = model_fit.predict(start=tsdTP.shape[0]-3, end=tsdTP.shape[0]+2)
    plt.plot(tsdTP)
    plt.plot(pred)
    print(plt.show())